In [6]:
#Author: Jonathan Ehring
#Class: MEMT680 Data Analysis and Machine Learning
#Instructor: Prof. Joshua Agar



import random
import time
#--------------------------------------------------------------------------------------------------------------#
#Parent class
class Dice:
    def __init__(self):
        self.die1 = 0
        self.die2 = 0
        self.total = 0
#--------------------------------------------------------------------------------------------------------------#       
    #This method rolls the dice using random integers. Returns 3 values, die1, die2 and die1 + die2
    def rollDice(self):
        self.die1 = random.randint(1,6)
        self.die2 = random.randint(1,6)
        self.total = self.die1 + self.die2
        return self.die1, self.die2, self.total
#----------------------------------------------------------------------------------------------------------------------#
#Child class of Dice
class Table(Dice):
    def __init__(self):
        super().__init__()
        self.Point = False #the point is initialized to False since a point has not yet been set
#----------------------------------------------------------------------------------------------------------------------#
#Child class of Table and Dice
class Player(Table):
    def __init__(self):
        super().__init__()
        self.name = input("Please enter the player name: ") #get the player's name
        self.bankroll = 0 #initialize bankroll
        
        #try-except statement to get bankroll from the player
        while True:
            try:
                self.bankroll = int(input("Please enter your bankroll (buy in): $"))
                assert(self.bankroll>0) #bankroll must be a positive integer value
                break
            except:
                print("Please enter a valid bankroll")
        
#----------------------------------------------------------------------------------------------------------------------#
#Child class of Player, Table and Dice
class Bets(Player):
    
    def __init__(self):
        super().__init__()
        #initialize values
        self.bet = 0
        self.passBet = 0
        self.doNotPassBet = 0
        self.oddsBet = 0
        self.point = 0
        self.gameOn = True
        self.bet_loss_type = ""
        self.bet_loss_amount = 0
#--------------------------------------------------------------------------------------------------------------#                
    #checks to ensure the player has enough money to play
    def insufficent_funds(self):
        while True:
            if self.bet > self.bankroll:
                try: 
                    self.bet = int(input("Insufficient funds: Player " + self.name + " has a bankroll of $"+str(self.bankroll)+". Please enter a new bet: $"))
                    assert(self.bet <= self.bankroll) #bet must be less than bankroll
                    assert(self.bet > 0) # bet must be greater than 0
                    break
                except:
                    print("Insufficient funds or invalid bet: Player " + self.name + " has a bankroll of $"+str(self.bankroll)+". Please increase bankroll to play")
            else:
                break
#--------------------------------------------------------------------------------------------------------------#    
    #prints the main menu for the user to select options
    def printMainMenu(self):
        print("------------------MAIN MENU--------------------------")
        print("1) Pass Line Bet")
        print("2) Do Not Pass Line Bet")
        print("3) Increase buy in")
        print("4) Print bankroll")
        print("5) Cash out & Walk Away")
#--------------------------------------------------------------------------------------------------------------#
    #Initiates the pass line bet
    def PassLineBet(self):
        self.passBet = self.ingestBet()#gets the bet from the player
        self.shooter()#calls the shooter method to roll the dice
        #if the shooter rolls a 2, 3 or 12 the player loses
        if self.total == 2 or self.total == 3 or self.total == 12:
            self.bet_loss("pass line bet", self.passBet) #player loss
            self.Point = False #no point is established
        #if the shooter rolls a 7 or 11 the player wins   
        elif self.total == 7 or self.total == 11:
            self.bet_won("pass line bet", self.passBet)#player wins
            self.Point = False #no point is established
        #set the point
        else:
            time.sleep(1)
            print("POINT IS SET TO " + str(self.total)) 
            self.point = self.total
            self.Point = True
        #continue to roll the dice and take bets while the point is set
        while self.Point:
            #Make sure the player has enough money to play odds
            if (self.bankroll - self.passBet) == 0:
                time.sleep(1)
                print("------------------------------------------")
                print("Odds is not available since bankroll is $0")
                print("------------------------------------------")
            #player has enough money...ask player if they would like to play odds
            else:
                self.Odds()
            
            #since the point is set, the player loses if the shooter rolls a 7
            if self.total == 7:
                self.bet_loss("pass line bet", self.passBet) #player loses
                time.sleep(1)
                self.Point = False
            #player wins if the shooter rolls the point
            elif self.total == self.point:
                self.bet_won("pass line bet", self.passBet)
                self.Point = False
        #reconcile all bets and print the bankroll
        self.Reconcile()
#--------------------------------------------------------------------------------------------------------------#
    def DoNotPassLineBet(self):
        #get bet from the user
        self.doNotPassBet = self.ingestBet()
        self.shooter() #roll the dice
        #player loses if the shooter rolls a 7 or 11
        if self.total == 7 or self.total == 11:
            self.bet_loss("do not pass line bet", self.doNotPassBet) #player loses
            self.Point = False
        #player wins if the shooter rolls a 2,3 or 12
        elif self.total == 2 or self.total == 3 or self.total == 12:
            self.bankroll += self.doNotPassBet
            self.bet_won("do not pass line bet", self.doNotPassBet) #player wins
            self.Point = False
        #set the point
        else:
            print("POINT IS SET TO " + str(self.total))
            self.point = self.total
            self.Point = True
        #continue to roll the dice and take bets while the point is set
        while self.Point:
            self.shooter() #roll dice
            #player wins if the shooter rolls a 7 after the point is set
            if self.total == 7:
                self.bet_won("do not pass line bet", self.doNotPassBet) #player wins
                self.Point = False
            #player loses if the shooter rolls the point
            elif self.total == self.point:
                self.bet_loss("do not pass line bet", self.doNotPassBet) #player loses
                self.Point = False
        self.Reconcile() #reconcile bets and print the players bankroll
#--------------------------------------------------------------------------------------------------------------#  
    #This method allows the player to select from the menu choices:
    # 1) Pass line bet
    # 2) Do not pass line bet
    # 3) Increase bankroll/buy in
    # 4) Print bankroll
    # 5) Cash out/walk away/exit game
    def bettingTurn(self):
        #get choice from player
        choice = int(input())
        #pass line bet
        if choice == 1:
            #ensure the player has enough money to continue
            if self.bankroll <= 0:
                print("Please increase bankroll before playing")
                bets.printMainMenu()
                bets.bettingTurn()
            #the player has enough money, continue
            else:
                bets.PassLineBet()
        #do not pass line bet
        elif choice == 2:
            #ensure the player has enough money to continue
            if self.bankroll <= 0:
                print("Please increase bankroll before playing")
                bets.printMainMenu()
                bets.bettingTurn()
            #the player has enough money, continue
            else:
                bets.DoNotPassLineBet()
        #increase bankroll
        elif choice == 3:
            self.bankroll += int(input("Enter new buy in: $"))
            print("Your bankroll is now: $"+str(self.bankroll))
            #go back to the start menu
            bets.printMainMenu()
            bets.bettingTurn()
        #print your bankroll
        elif choice == 4:
            print("Your bankroll is: $"+str(self.bankroll))
            #go back to the start menu
            bets.printMainMenu()
            bets.bettingTurn()
        #exit game
        elif choice == 5:
            bets.gameOn = False
            print("Thanks for playing! Your bankroll is: $"+str(self.bankroll))
#--------------------------------------------------------------------------------------------------------------#
    #This method gets the bet from the user and returns the bet as an integer
    def ingestBet(self):
        self.bet = int(input("Enter Bet: $")) #only accepts integer values
        self.insufficent_funds() #check if the player has sufficient funds to place bet
        return self.bet #return bet
#--------------------------------------------------------------------------------------------------------------#
    #This method checks if a bet has been made
    def bet_made(self):
        if self.passBet > 0:
            print("Pass Line Bet: $"+str(self.passBet))
        if self.doNotPassBet > 0:
            print("Don't Pass Line Bet: $"+str(self.doNotPassBet))
#--------------------------------------------------------------------------------------------------------------#
    #This method prints the bet type and amount loss
    def bet_loss(self, betType, betAmount):
        self.bankroll -= betAmount #update the bankroll
        print("--------------------------------------")
        print("| You loss: "+betType+" | "+"Amount: $"+str(betAmount) + " |")
        print("--------------------------------------")
#--------------------------------------------------------------------------------------------------------------# 
    #This method prints the bet type and amount won
    def bet_won(self, betType, betAmount):
        self.bankroll += betAmount #update the bankroll
        print("------------------------------------")
        print("| You Win: " + betType + " | "+"Amount: $"+str(betAmount) + " |")
        print("------------------------------------")
#--------------------------------------------------------------------------------------------------------------#
    #This method rolls the dice
    def shooter(self):
        if self.passBet > 0 or self.doNotPassBet > 0: #checks that the bet entered is greater than 0
            time.sleep(1)
            D1, D2, total = Dice.rollDice(self)
            print("Rolling dice... " + str(D1) + " and " +str(D2))
#--------------------------------------------------------------------------------------------------------------#  
    #This method handles the odds bets placed during the pass line bet
    def Odds(self):
        playOdds = False #flag to play odds until set to true
        valid = False #flag to handle valid player input
        
        #continue playing until true
        while playOdds == False:
            choice = (input("Would you like to play odds? (y/n)")) #get user input
            
            if choice == 'y': #player chooses to play odds
                
                while valid == False: 
                    #get odds bet from user. This is the total odds bet so that if the player wants to update their 
                    #odds bet, it does not build off previous odds bets
                    self.oddsBet = int(input("Enter total odds bet (Max is 5x): $"))
                    #check that the odds bet does not go over 5 times the pass line bet
                    if self.oddsBet > (5*self.passBet):
                        print("You've reached the max bet. Please decrease bet below 5x")
                    elif self.oddsBet > self.bankroll: #check that the player does not exceed their bankroll
                        print("You've exceeded your bankroll")
                    else:
                        valid = True
            
                playOdds = True #player has enough money to play and has not exceeded maximum allowable odds bet
                self.shooter() #roll dice
                #player wins odds 2:1 on 4 and 10
                if self.total == self.point == 4 or self.total == self.point == 10:
                    self.bet_won("odds bet", self.oddsBet*2) #player wins
                #player wins odds 3:2 on 5 and 9
                elif self.total == self.point == 5 or self.total == self.point == 9:
                    self.bet_won("odds bet", self.oddsBet*(3/2)) #player wins
                #player wins odds 6:5 on 6 and 8
                elif self.total == self.point == 6 or self.total == self.point == 8:
                    self.bet_won("odds bet", self.oddsBet*(6/5)) #player wins
                elif self.total == 7: #loses on a 7
                    self.bet_loss("odds bet", self.oddsBet) #player loses
            
            #player chooses not to play odds    
            elif choice == 'n':
                print("")
                playOdds = True
            #player did not enter y or n, so the prompt is printed again
            else:
                print("Please enter 'y' for yes or 'n' for no")
#--------------------------------------------------------------------------------------------------------------#
    #returns the bankroll at the end of the round
    def Reconcile(self):
        print("----Reconcile Bets------")
        print("Your bankroll is now: $"+str(self.bankroll))
        
#--------------------------------------------------------------------------------------------------------------#
#-------------------------------------------------MAIN---------------------------------------------------------#
#--------------------------------------------------------------------------------------------------------------#
#Print the main menu
print("------------------------------------------")
print("      Welcome to the game of craps")
print("------------------------------------------")
print("Starting game ...")
time.sleep(1)

#instantiate Bets class as bets object
bets = Bets()

#continue playing the game until the player selects 5) Cash Out
while bets.gameOn:
    bets.printMainMenu() #prints the main menu with 5 choices
    bets.bettingTurn() #initiates choice selection

#--------------------------------------------------------------------------------------------------------------#
#--------------------------------------------END OF PROGRAM----------------------------------------------------#
#--------------------------------------------------------------------------------------------------------------#


    




------------------------------------------
      Welcome to the game of craps
------------------------------------------
Starting game ...
Please enter the player name: Jonathan
Please enter your bankroll (buy in): $10
------------------MAIN MENU--------------------------
1) Pass Line Bet
2) Do Not Pass Line Bet
3) Increase buy in
4) Print bankroll
5) Cash out & Walk Away
1
Enter Bet: $10
Rolling dice... 5 and 3
POINT IS SET TO 8
------------------------------------------
Odds is not available since bankroll is $0
------------------------------------------
------------------------------------
| You Win: pass line bet | Amount: $10 |
------------------------------------
----Reconcile Bets------
Your bankroll is now: $20
------------------MAIN MENU--------------------------
1) Pass Line Bet
2) Do Not Pass Line Bet
3) Increase buy in
4) Print bankroll
5) Cash out & Walk Away
3
Enter new buy in: $100
Your bankroll is now: $120
------------------MAIN MENU--------------------------
1) Pass L